# Audio Recorder

This notebook demonstrates how to use `Preparation.Import.audio_recorder` to list input devices and record short audio snippets organized by defect type.

Notes:
- Ensure you installed the dependencies from `Preparation/Import/requirements.txt`.
- On macOS grant microphone permission to your terminal/IDE before recording.

In [ ]:
import os

# Clone the repository if it doesn't exist
if not os.path.exists('fred-predictive-maintenance'):
    !git clone https://github.com/Ibrahimhk98/FrED-Predictive-Maintenance-.git
else:
    print("Repository already cloned.")

In [ ]:
import subprocess
import sys

# Install required dependencies

def install_requirements():
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "/requirements.txt"])
        print("Requirements installed successfully!")
    except subprocess.CalledProcessError as e:
        print(f"Error installing requirements: {e}")
    except FileNotFoundError:
        print("Requirements file not found. Please ensure /requirements.txt exists.")

install_requirements()

In [ ]:
# Example: list devices and record a 2 second snippet
from Preparation.Import.audio_recorder import list_audio_devices, record_snippet
# Build and display the recorder UI from the audio_recorder module
from Preparation.Import.audio_recorder import create_recorder_ui
from google.colab import drive
import os

# List available input devices (show index and name)
devices = list_audio_devices()
print('\n'.join(devices))
# Mount Google Drive
drive.mount('/content/drive')

# Use the shared drive provided by instructor
data_folder = "/content/drive/Shareddrives/FrED_PdM/"
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
    print(f"Created folder: {data_folder}")
else:
    print(f"Folder already exists: {data_folder}")

# Create recorder UI with Google Drive data folder
ui = create_recorder_ui(base_dir=data_folder)
display(ui)


0: Ibrahim’s iPhone Microphone
2: MacBook Pro Microphone
4: Immersed


In [2]:
# Record test files in different directory
ui = create_recorder_ui(base_dir="sample_data/audio")
display(ui)